In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from utils.Config import Config

with open("config.json", "r") as f:
    cfg_json = json.load(f)
    cfg = Config(**cfg_json)

In [3]:
import pickle as pkl
from dataset.dataset import SaladsDataset

with open("../data/pickles/50_salads_unified.pkl", "rb") as f:
    dataset = pkl.load(f)

salads_dataset = SaladsDataset(dataset['target'], dataset['stochastic'])

In [4]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(salads_dataset, train_size=0.7, shuffle=True, random_state=42)

In [5]:
from utils.pm_utils import discover_dk_process, remove_duplicates_dataset, pad_to_multiple_of_n
from utils.graph_utils import prepare_process_model_for_gnn, get_process_model_reachability_graph_transition_matrix

dk_process_model, dk_init_marking, dk_final_marking = discover_dk_process(train_dataset, cfg, preprocess=remove_duplicates_dataset)

In [6]:
pm_nx_data = prepare_process_model_for_gnn(dk_process_model, dk_init_marking, dk_final_marking, cfg)

C:\Users\Maxim\anaconda3\Lib\site-packages\torch_geometric\utils\convert.py:278: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  data_dict[key] = torch.as_tensor(value)


In [7]:
from modules.GraphEncoder import GraphEncoder

g_enc = GraphEncoder(pm_nx_data.num_nodes, 128, 128, 128, pooling=None).to(cfg.device).float()
g_enc_add = GraphEncoder(pm_nx_data.num_nodes, 128, 128, 128).to(cfg.device).float()

In [8]:
pm_nx_data = pm_nx_data.to(cfg.device)

In [9]:
out_g = g_enc(pm_nx_data)

In [10]:
out_g_add = g_enc_add(pm_nx_data)

In [11]:
out_g.shape

torch.Size([73, 128])

In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=8
    )

dummy_x, dummy_y = (w.permute(0, 2, 1).to(cfg.device).float() for w in next(iter(train_loader)))

In [13]:
from ddpm.ddpm_multinomial import Diffusion

diffuser = Diffusion(noise_steps=cfg.num_timesteps, device=cfg.device)

In [26]:
from denoisers.ConditionalUnetGraphDenoiser import ConditionalUnetGraphDenoiser

graph_denoiser = ConditionalUnetGraphDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes, max_input_dim=salads_dataset.sequence_length, graph_data=pm_nx_data, num_nodes=pm_nx_data.num_nodes, embedding_dim=128, hidden_dim=128, pooling=None).to(cfg.device)

In [27]:
t = diffuser.sample_timesteps(dummy_x.shape[0]).to(cfg.device)

In [28]:
graph_denoiser(dummy_x, t, None, None)[0].shape

RuntimeError: shape '[3, 64, -1]' is invalid for input of size 4672